# Load packages

In [37]:
import pandas as pd
import re # regexes
from pathlib import Path # better way to work with filepaths

# Load in Data

In [29]:
# List all .txt files in data directory
# One file for each year of baby names in the US since late 1800s until 2018
baby_name_folder = Path.cwd() / "data" / "names"
baby_name_files = [x for x in baby_name_folder.glob('*txt')]
# Have a look at one file
baby_name_files[0]

WindowsPath('C:/Users/user457/Documents/udacity-data-scientist-nanodegree-project-1-write-a-data-science-blog-post/data/names/yob1880.txt')

In [54]:
def parse_raw_data(f):
    df = pd.read_csv(f, header = 0, names=['name','sex','babies'])
    # Add year from the filename
    # Data itself doesn't have the year, but the filename always has the pattern of yob{4-digit-Year}.txt
    df["year"] = int(re.search(r'\d\d\d\d', str(f), re.IGNORECASE).group(0))
    return(df)

# Map function to each file
baby_names_list = map(parse_raw_data, baby_name_files)
# Load all data into a single pandas dataframe
baby_names = pd.concat(baby_names_list)
baby_names.head()

,name,sex,babies,year
0,Anna,F,2604,1880
1,Emma,F,2003,1880
2,Elizabeth,F,1939,1880
3,Minnie,F,1746,1880
4,Margaret,F,1578,1880


# Exploratory data analysis 

### How many years of data do we have?

In [55]:
baby_names["year"].describe()

count    1.956907e+06
mean     1.975565e+03
std      3.419002e+01
min      1.880000e+03
25%      1.952000e+03
50%      1.985000e+03
75%      2.004000e+03
max      2.018000e+03
Name: year, dtype: float64

In [56]:
# Avoid scientific notation
# https://stackoverflow.com/questions/40347689/dataframe-describe-suppress-scientific-notation
baby_names["year"].describe().apply(lambda x: format(x, 'f'))

count    1956907.000000
mean        1975.564815
std           34.190022
min         1880.000000
25%         1952.000000
50%         1985.000000
75%         2004.000000
max         2018.000000
Name: year, dtype: object

In [62]:
baby_names['year'].nunique()

139

* 1880 to 2018, 139 years of data

### How has the US born population increased over the years - using baby names as a proxy?

In [59]:
baby_names.groupby('year')['babies'].sum()

year
1880     194419
1881     185777
1882     213385
1883     208932
1884     234244
         ...   
2014    3677978
2015    3671535
2016    3638301
2017    3542175
2018    3468665
Name: babies, Length: 139, dtype: int64

* We can see the huge growth over the decades. Remember, this is NOT the total US Population.

### Has the diversity (uniqueness) of baby names increased over the years?

In [63]:
baby_names.groupby('year')['name'].nunique()

year
1880     1889
1881     1830
1882     2012
1883     1962
1884     2158
        ...  
2014    30742
2015    30603
2016    30415
2017    30006
2018    29494
Name: name, Length: 139, dtype: int64

* Appears that is the case

### What first letter is the most popular in the US in the past few years?

In [90]:
baby_names['first_letter'] = baby_names['name'].str[0]
ag = aggregate(babies ~ year * first_letter , data = baby_names, FUN = sum)
#yearly_first_letters = baby_names.groupby(by=['year','first_letter'])['babies'].sum()
#yearly_first_letters = yearly_first_letters.sort_values(['year', 'first_letter'], ascending=[False,False])
#yearly_first_letters.loc[ [2008, 2018] , : ]

SyntaxError: invalid syntax (<ipython-input-90-2a57165b774f>, line 2)